# File Formats

Files and file formats are a very important idea for digital curation because they are the primary abstraction we use to talk about data. If you open up your File Explorer on your computer you can see your operating system and applications are all made up of files. Our so called "smart" phones are often dumbed down to prevent us from seeing the files that are there. Even on the web where data feels like it is "in the cloud" there are files on someone else's computer that we call a server or perhaps thousands of servers that make files available over the network as bit streams. Bit streams is another name for a file, a discrete sequence of binary data.

But as we read about in Chapter 3 of The Theory and Craft of Digital Preservation, these files, or bit streams, have something in common: they all have a particular *format*. These formats define the shape of the data, and often have *standards* associated with how the files of a particular format should be put together. Having standard file formats allows us to reliably *render* the files in particular ways. For example the MP4 file format can be rendered by a video player like [VLC](https://www.videolan.org/vlc/) and also your web browser. Agreement about how an MP4 should be structured is what allows multiple applications to open an MP4 file, play it, pause it, adjust its volume, etc.

In this notebook we will explore two ways of discovering what format a file is in, and how to determine what application can be used to render it.

## File Extensions

As we saw in the Module 2 Notebook assignment many files have a file extension, which gives us a hint about what format the data is in. This is typically a short sequence of characters following a period in the filename. File extensions are a naming convention that may or may not correctly identify the format that the file is in. For example the filename:

    paper.pdf
    
has the file extension `pdf` which is an acronym for the [Portable Document Format](https://en.wikipedia.org/wiki/PDF) that was created by Adobe Corporation in 1993 and was standardized as [ISO 32000](https://www.iso.org/standard/51502.html) in 2008. But if you wanted you could rename a `file.mp4` to a `file.pdf`. That woudln't change the format of the file itself, just its name.

Lets take another look at some of the file extensions. We will start by downloading the files we used in Module 2. that are part of the [Digital Corpora](https://digitalcorpora.org/) dataset.

In [71]:
! pip3 install --quiet git+https://github.com/edsu/inst341data

In [41]:
import inst341data
inst341data.get_module_2('inst341')

Downloaded inst341


This downloaded our test data and put it into a directory named `inst341` in our Jupyter Notebook's working directory. Remember we can use Python's [pathlib] module to interact with the file system and list the contents of this directory and print out the file extensions.

In [9]:
import pathlib

data = pathlib.Path('inst341')

for path in data.iterdir():
    print(path, path.suffix)

inst341/710097.pdf .pdf
inst341/481368.csv .csv
inst341/368751.html .html
inst341/951980.gz .gz
inst341/789265.ppt .ppt
inst341/512608.ppt .ppt
inst341/377087.txt .txt
inst341/447656.html .html
inst341/362088.pdf .pdf
inst341/356028.pdf .pdf
inst341/278141.jpg .jpg
inst341/033333.pdf .pdf
inst341/098807.doc .doc
inst341/115389.jpg .jpg
inst341/441236.ps .ps
inst341/286538.pdf .pdf
inst341/120637.ppt .ppt
inst341/064568.pdf .pdf
inst341/306840.html .html
inst341/925740.doc .doc
inst341/761213.ps .ps
inst341/763624.html .html
inst341/116114.dbase3 .dbase3
inst341/215842.txt .txt
inst341/837467.jpg .jpg


We can use a [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) from the [collections](https://docs.python.org/3/library/collections.html) module to count up how many files of each type there are:

In [24]:
from collections import Counter

extension_counts = Counter()

for path in data.iterdir():
    extension_counts[path.suffix] += 1
    
print(extension_counts)

Counter({'.pdf': 6, '.html': 4, '.ppt': 3, '.jpg': 3, '.txt': 2, '.doc': 2, '.ps': 2, '.csv': 1, '.gz': 1, '.dbase3': 1})


## Magic File

A more accurate way of identifying the format of a file is to look for what are called *magic numbers* that are in the file. You can think of these as bits of data in the file that uniquely identify the file format--kind of like a fingerprint for the file. You can see a list of these types of signatures in [this](https://en.wikipedia.org/wiki/List_of_file_signatures) Wikipedia article.

Instead of you having to look for these signatures yourself fortunately there is a Python extension called [python-magic](https://pypi.org/project/python-magic/) that will do the searching for you. The two commands below will install it into your notebook environment.

In [ ]:
! sudo apt-get --quiet install -q libmagic1
! pip -q install python-magic

With python-magic installed we can now import it and use it to identify the format of the file `inst341/710097.pdf`:

In [38]:
import magic

magic.from_file('inst341/710097.pdf')

'PDF document, version 1.3'

So now we know that the file is actually a PDF, and we even know the version number. The version is important because it may indicate that a particular version of a PDF viewing application will be needed to render it properly. Here is the version of another file:

In [32]:
magic.from_file('inst341/356028.pdf')

'PDF document, version 1.4'

The same is true of JPEG files which usually get a file extension of `.jpg`. Here are two different JPG files which have different version numbers. We can also see that the size of the images are different. We will be taking a closer look at internal metadata in the next module. But for now just notice how python-magic helps us see how different these files are:

In [39]:
magic.from_file('inst341/115389.jpg')

'JPEG image data, JFIF standard 1.01, aspect ratio, density 1x1, segment length 16, baseline, precision 8, 320x240, components 3'

In [40]:
magic.from_file('inst341/278141.jpg')

'JPEG image data, JFIF standard 1.02, resolution (DPI), density 72x72, segment length 16, comment: "File written by Adobe Photoshop\\250 5.0", baseline, precision 8, 180x234, components 3'

You can collect all the file formats and count them to get a sense of what types of data we have. Note that the `magic.from_file` function wants to be given a *string* so when using `pathlib` we have to call a Path objects `as_posix()` method to get the Path as a string.


In [51]:
formats = []

for p in data.iterdir():
    file_format = magic.from_file(p.as_posix())
    formats.append(file_format)

One easy way to count things in Python is to use the [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) class:

In [54]:
from collections import Counter

format_counts = Counter(formats)

for format, count in format_counts.most_common():
    print(count, format)


4 PDF document, version 1.3
2 ASCII text
1 HTML document, UTF-8 Unicode text, with very long lines, with CRLF, LF line terminators
1 gzip compressed data, was "srini_collisions96.ps", last modified: Wed Jan 22 14:45:32 1997, from Unix, original size modulo 2^32 754717
1 PDF document, version 1.6
1 HTML document, ASCII text, with very long lines, with CRLF, LF line terminators
1 Composite Document File V2 Document, Little Endian, Os: Windows, Version 5.1, Code page: 1252, Title: NACE Direct Assessment Standards, Author: LGoldberg, Template: Slit, Last Saved By: LGoldberg, Revision Number: 40, Name of Creating Application: Microsoft PowerPoint, Total Editing Time: 12:34:49, Create Time/Date: Fri Oct 24 20:01:47 2003, Last Saved Time/Date: Mon Nov  3 19:52:13 2003, Number of Words: 786
1 JPEG image data, JFIF standard 1.01, resolution (DPI), density 72x72, segment length 16, baseline, precision 8, 181x120, components 3
1 FoxBase+/dBase III DBF, 50 records * 224, update-date 99-7-13, at of

## PRONOM

[PRONOM](http://www.nationalarchives.gov.uk/PRONOM/) is a registry of file formats that is maintained by the UK National Archives. One nice thing about PRONOM is that they have assigned each format a unique identifier (PUID) which can be used to look up information about the format by [searching by the PUID](http://www.nationalarchives.gov.uk/PRONOM/PUID/proPUIDSearch.aspx?status=new).

The [puid](https://github.com/edsu/puid) module is a small utility that lets you look up the PUID for a file once it is installed.

In [56]:
! pip install --quiet puid

In [58]:
from puid import get_puid

get_puid('inst341/278141.jpg')

'fmt/44'

When you look up *fmt/44* in the PRONOM database you get a page that lists lots of information about the format. PRONOM also mentions when formats are deemed at risk and can help you figure out what formats to migrate to. We will be learning about migrating formats in the next module.

> The JPEG File Interchange Format (JFIF) is a file format for storing JPEG-compressed raster images. It was developed by the Independent JPEG Group and C-Cube Microsystems, in the absence of any such format being defined in the JPEG standard, and rapidly became a de facto standard. A JFIF file comprises a JPEG data stream together with a JFIF marker. It begins with a Start of Image (SOI) marker, immediately followed by a JFIF Application (APP0) marker and one or more optional application extension markers. This is followed by the JPEG image data, which is terminated by an End of Image (EOI) marker. JFIF supports up to 24-bit colour and uses lossy compression (based on the Discrete Cosine Transform algorithm). Other types of compression are available through JPEG extensions, including progressive image buildup, arithmetic encoding, variable quantization, selective refinement, image tiling, and lossless compression, but these may not be supported by all JFIF readers and writers.

## Exercise


### 1. Get Data

First download your module 3 data by replacing USERNAME in the string below with your UMD username (the same one you used in the Module 2 notebook).

In [ ]:
import inst341data

inst341data.get_module_3('USERNAME')

If that generated an error make sure you run the cell above that does the:

    pip install --quiet inst341data
    
Note: you may notice that your files do not have file extensions. It's not uncommon for file extensions to go missing, use an alternate form, or be incorrect. That's why file format identification tools are useful!
    
### 2. Format Counts

Use the **python-magic** module to determine what the most common file format in your directory. Please use the cell below for your Python code.

### 3. PUID Counts

Use the **puid** module to print out the PUID file format identifier counts for your files. Please use the cell below for your Python code.

### 4. Compare

Use the cell below to compare the **python-magic** and **puid** results. Why do you think the counts by format might be different?

### 5. Look Up

Look up the most commmon PUID in [PRONOM](http://www.nationalarchives.gov.uk/PRONOM/PUID/proPUIDSearch.aspx?status=new). What information do you see about it? Who developed the standard?